In [1]:
!nvidia-smi

Tue May 13 00:37:44 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.144.03             Driver Version: 550.144.03     CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce GTX 1650        Off |   00000000:01:00.0 Off |                  N/A |
| N/A   51C    P8              1W /   50W |       6MiB /   4096MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
!unzip /content/dataset.zip

unzip:  cannot find or open /content/dataset.zip, /content/dataset.zip.zip or /content/dataset.zip.ZIP.


In [3]:
import pandas as pd

In [ ]:
imbalance_data = pd.read_csv("/home/dhanu/NLP-proj1/artifacts/05_13_2025_02_22_05/DataIngestionArtifacts/imbalanced_data.csv")

FileNotFoundError: [Errno 2] No such file or directory: '/content/imbalanced_data.csv'

In [ ]:
imbalance_data.head()

## EDA

In [ ]:
import seaborn as sns

In [ ]:
sns.countplot(x='label', data=imbalance_data)

 - 0 ---> No hate
 - 1 ---> Hate

In [ ]:
imbalance_data.shape

In [ ]:
imbalance_data.isnull().sum()

In [ ]:
imbalance_data.drop("id", axis=1, inplace=True)

In [ ]:
imbalance_data.head()

In [ ]:
raw_data = pd.read_csv("/home/dhanu/NLP-proj1/artifacts/05_13_2025_02_22_05/DataIngestionArtifacts/raw_data.csv")

In [ ]:
raw_data.head()

In [ ]:
raw_data.shape

In [ ]:
raw_data.isnull().sum()

In [ ]:
# Let's drop the columns which are not required for us.
raw_data.drop(['Unnamed: 0','count','hate_speech','offensive_language','neither'],axis=1,inplace =True)

In [ ]:
raw_data.head()

In [ ]:
# Let's check for the unique values in the dataset
raw_data['class'].unique()

In [ ]:
# Plotting the countplot for our new dataset
sns.countplot(x='class',data =raw_data)

- class 0: hate
- class 1: abusive
- class 2: no hate

In [ ]:
# Let's copy the valus of the class 1 into class 0.
raw_data[raw_data['class']==0]['class']=1

In [ ]:
raw_data.head()

In [ ]:
raw_data['class'].unique()

In [ ]:
# Let's check the values in the class 0
raw_data[raw_data['class']==0]

In [ ]:
# replace the value of 0 to 1
raw_data["class"].replace({0:1},inplace=True)

In [ ]:
raw_data["class"].unique()

In [ ]:
sns.countplot(x="class",data= raw_data)

In [ ]:
# Let's replace the value of 2 to 0.
raw_data["class"].replace({2:0}, inplace = True)

In [ ]:
sns.countplot(x='class',data=raw_data)

In [ ]:
imbalance_data.head()

In [ ]:
raw_data.head()

In [ ]:
# Let's change the name of the 'class' to label
raw_data.rename(columns={'class':'label'},inplace =True)

In [ ]:
raw_data.head()

In [ ]:
# Let's concatinate both the data into a single data frame.
frame = [imbalance_data, raw_data]
df = pd.concat(frame)

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
sns.countplot(x='label',data=df)

## Preprocessing

In [ ]:
import re
import nltk
import string
from nltk.corpus import stopwords
nltk.download('stopwords')

In [ ]:
# Let's apply stemming and stopwords on the data
stemmer = nltk.SnowballStemmer("english")
stopword = set(stopwords.words('english'))

In [ ]:
# Let's apply regex and do cleaning.
def data_cleaning(words):
    words = str(words).lower()
    words = re.sub('\[.*?\]', '', words)
    words = re.sub('https?://\S+|www\.\S+', '', words)
    words = re.sub('<.*?>+', '', words)
    words = re.sub('[%s]' % re.escape(string.punctuation), '', words)
    words = re.sub('\n', '', words)
    words = re.sub('\w*\d\w*', '', words)
    words = [word for word in words.split(' ') if words not in stopword]
    words=" ".join(words)
    words = [stemmer.stem(words) for word in words.split(' ')]
    words=" ".join(words)

    return words

In [ ]:
df["tweet"][1]

In [ ]:
# let's apply the data_cleaning on the data.
df['tweet']=df['tweet'].apply(data_cleaning)

In [ ]:
df["tweet"][1]

In [ ]:
x = df['tweet']
y = df['label']

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# Let's split the data into train and test
x_train,x_test,y_train,y_test = train_test_split(x,y, random_state = 42)

print(len(x_train),len(y_train))
print(len(x_test),len(y_test))

## Feature engineering

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences

In [ ]:
max_words = 50000
max_len = 300

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(x_train)

sequences = tokenizer.texts_to_sequences(x_train)
sequences_matrix = pad_sequences(sequences,maxlen=max_len)

In [ ]:
sequences_matrix

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding, SpatialDropout1D
from keras.optimizers import RMSprop

In [ ]:
# Creating model architecture.
model = Sequential()
model.add(Embedding(max_words,100,input_length=max_len))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100,dropout=0.2,recurrent_dropout=0.2))
model.add(Dense(1,activation='sigmoid'))
model.summary()


In [ ]:
model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])

In [ ]:
# starting model training
history = model.fit(sequences_matrix,y_train,batch_size=128,epochs = 1,validation_split=0.2)

In [ ]:
test_sequences = tokenizer.texts_to_sequences(x_test)
test_sequences_matrix = pad_sequences(test_sequences,maxlen=max_len)

In [ ]:
test_sequences_matrix

In [ ]:
# Model evaluation
accr = model.evaluate(test_sequences_matrix,y_test)

In [ ]:
lstm_prediction = model.predict(test_sequences_matrix)

In [ ]:
res = []
for prediction in lstm_prediction:
    if prediction[0] < 0.5:
        res.append(0)
    else:
        res.append(1)

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
print(confusion_matrix(y_test,res))

In [ ]:
import pickle
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# Let's save the mdoel.
model.save("model.h5")

In [ ]:
import keras

In [ ]:
load_model=keras.models.load_model("model.h5")
with open('tokenizer.pickle', 'rb') as handle:
    load_tokenizer = pickle.load(handle)

In [ ]:
# Let's test our model on custom data.
test = 'i love this movie'

def clean_text(text):
    print(text)
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    print(text)
    text = [word for word in text.split(' ') if word not in stopword]
    text=" ".join(text)
    text = [stemmer.stem(word) for word in text.split(' ')]
    text=" ".join(text)
    return text

test=[clean_text(test)]
print(test)

seq = load_tokenizer.texts_to_sequences(test)
padded = pad_sequences(seq, maxlen=300)
print(seq)

pred = load_model.predict(padded)

print("pred", pred)
if pred<0.5:
    print("no hate")
else:
    print("hate and abusive")
